<a href="https://colab.research.google.com/github/Vishnu-N-07/AI/blob/main/LLM_finetune_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TuneKit: Fine-Tune Llama 3.2 3B

## Dataset Overview
| Metric | Value |
|--------|-------|
| **Model** | Llama 3.2 3B (`meta-llama/Llama-3.2-3B-Instruct`) |
| **Examples** | 103 conversations |
| **Task Type** | Conversation |
| **Estimated Time** | ~5 minutes |

> **Note:** `meta-llama/Llama-3.2-3B-Instruct` is a gated model. You need to:
> 1. Accept the license at [huggingface.co/meta-llama/Llama-3.2-3B-Instruct](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct)
> 2. Login with your HuggingFace token (Cell 3 below)

## Before You Start
1. **GPU Required**: Go to `Runtime` > `Change runtime type` > Select **T4 GPU**
2. **Run All Cells**: Click `Runtime` > `Run all` or press `Ctrl+F9`
3. **Download Model**: After training, download your fine-tuned model from the Files panel

## Why Unsloth?
- **2-5x faster training** with optimized kernels
- **70% less memory** usage
- **Native 4-bit quantization** for efficiency
- **Easy export** to GGUF, 16-bit, 4-bit formats

---
*Generated by [TuneKit](https://github.com/riyanshibohra/TuneKit) - Fine-tuning made simple*


In [ ]:
%%capture
# Install Unsloth - 2x faster training, 70% less memory
!pip install unsloth


In [ ]:
# Verify GPU
import torch
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️ WARNING: No GPU! Go to Runtime > Change runtime type > T4 GPU")


## HuggingFace Login Required

**`meta-llama/Llama-3.2-3B-Instruct`** is a gated model. You must:
1. **Accept the license** at [huggingface.co/meta-llama/Llama-3.2-3B-Instruct](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct)
2. **Paste your token below** and run the cell

Get your token at: [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)


In [ ]:
#@title HuggingFace Login (Required for this model)
#@markdown Paste your HuggingFace token below. Get one at [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)
HF_TOKEN = ""  #@param {type:"string"}

if not HF_TOKEN or not HF_TOKEN.startswith("hf_"):
    raise ValueError("⛔ STOP: Please paste your HuggingFace token above (starts with 'hf_') and re-run this cell.")

from huggingface_hub import login
login(token=HF_TOKEN)
print("✅ Successfully logged in to HuggingFace!")


In [ ]:
#@title Load Training Dataset (click to expand)
import base64
import json

# Your dataset (embedded from TuneKit)
DATASET_B64 = """..."""

# Decode and parse
dataset_jsonl = base64.b64decode(DATASET_B64).decode()
conversations = [json.loads(line) for line in dataset_jsonl.strip().split("\n") if line.strip()]

print(f"Loaded {len(conversations):,} conversations")
print(f"\nSample conversation:")
if conversations:
    sample = conversations[0]
    for msg in sample.get("messages", [])[:3]:
        role = msg.get("role", "unknown")
        content = msg.get("content", "")[:100]
        print(f"  {role}: {content}...")


In [ ]:
# Load model with Unsloth (4-bit quantization for memory efficiency)
import os
os.environ["TORCHDYNAMO_DISABLE"] = "1"  # Disable dynamo to avoid compilation errors

from unsloth import FastLanguageModel
import torch

MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"
MAX_SEQ_LENGTH = 2048

print(f"Loading {MODEL_ID} with Unsloth...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_ID,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,  # Auto-detect (float16 for T4)
    load_in_4bit=True,  # 4-bit quantization
)

# Configure LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Rank - lower = smaller adapter, faster training
    lora_alpha=16,  # Scaling factor
    lora_dropout=0,  # No dropout for faster training
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    use_gradient_checkpointing="unsloth",  # Optimized checkpointing (4x longer context)
    random_state=42,
)

# Print trainable parameters
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Model loaded!")
print(f"Trainable parameters: {trainable:,} / {total:,} ({100*trainable/total:.2f}%)")


In [ ]:
# Prepare dataset for training
from datasets import Dataset

def format_conversation(example):
    """Format conversation using chat template."""
    messages = example.get("messages", [])

    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )

    return {"text": text}

# Create dataset
dataset = Dataset.from_list(conversations)
dataset = dataset.map(format_conversation, remove_columns=dataset.column_names)

print(f"Dataset prepared: {len(dataset)} examples")
print(f"\nSample formatted text (first 500 chars):")
print(dataset[0]["text"][:500] + "...")


In [ ]:
# Train with SFTTrainer (optimized for Unsloth)
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

training_args = TrainingArguments(
    output_dir="./results",

    # Training params
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,

    # Optimizer
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_steps=5,

    # Precision (auto-detect best for GPU)
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),

    # Logging
    logging_steps=10,

    # Saving
    save_strategy="epoch",
    save_total_limit=2,

    # Optimization
    optim="adamw_8bit",  # 8-bit Adam for memory efficiency
    lr_scheduler_type="linear",
    seed=42,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    packing=False,  # Set True for short sequences
    args=training_args,
)

# GPU stats before training
gpu_stats = torch.cuda.get_device_properties(0)
start_memory = round(torch.cuda.max_memory_reserved() / 1024**3, 2)
max_memory = round(gpu_stats.total_memory / 1024**3, 2)
print(f"GPU: {gpu_stats.name}")
print(f"Memory before: {start_memory} GB / {max_memory} GB")

print("\nStarting training...")
print("=" * 50)

trainer_stats = trainer.train()

# GPU stats after training
end_memory = round(torch.cuda.max_memory_reserved() / 1024**3, 2)
print("=" * 50)
print(f"Training complete!")
print(f"Peak GPU memory: {end_memory} GB ({round(end_memory/max_memory*100, 1)}% of {max_memory} GB)")


In [ ]:
# Save the fine-tuned model
import os

OUTPUT_DIR = "./fine_tuned_llama_3.2_3b"

# Save LoRA adapter
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"LoRA adapter saved to: {OUTPUT_DIR}")

# List saved files
files = os.listdir(OUTPUT_DIR)
print(f"\nFiles saved:")
for f in files:
    size = os.path.getsize(os.path.join(OUTPUT_DIR, f)) / 1024
    print(f"  {f} ({size:.1f} KB)")


In [ ]:
# Test your fine-tuned model
FastLanguageModel.for_inference(model)  # Enable faster inference

# System prompt from your training data
SYSTEM_PROMPT = """ROLE: Insurance Assistant ONLY

GOAL:
- Collect all required insurance quote fields and complete the flow correctly.

SUCCESS CONDITIONS:
- All fields in the schema list are collected or explicitly skipped.
- Exactly ONE question is asked per assistant message.
- A normalized summary is shown with [[CONFIRM_DETAILS]].
- User explicitly confirms the summary.
- Final message ends with [QUOTE_COMPLETE].

--------------------------------------------------
DOMAIN & GREETING CONTROL
--------------------------------------------------
- You are an INSURANCE assistant only.
- If a request is outside insurance, respond:
  "I am an insurance assistant and cannot help with general queries."
- Greet ONLY if insurance intent is not yet clear.
- If insurance intent is detected, SKIP greeting and start field collection immediately.

ALLOWED TOPICS:
- Greetings
- Vehicle insurance
- Policy, Policy Quote Collection, coverage, IDV

FORBIDDEN TOPICS:
- Coding
- Writing emails
- Travel
- Health advice
- Legal advice
- Translation
- premium calculation

--------------------------------------------------
INTENT DETECTION
--------------------------------------------------
Insurance intent keywords include:
insurance, insure, policy, vehicle, car, bike, truck

ON INSURANCE INTENT:
- Immediately enter field collection mode.
- Do NOT repeat greeting.
- Do NOT re-confirm intent.

STARTUP RULE:
- If insurance intent is detected and no fields are collected yet,
  ask the FIRST field (vehicle_type) immediately.

--------------------------------------------------
FIELD COLLECTION & EXTRACTION
--------------------------------------------------
- After EVERY user message, attempt to extract values for ALL unfilled fields.
- Always perform extraction BEFORE asking the next question.
- Follow field order strictly.
- A field may be skipped ONLY if the user explicitly says:
  "skip", "not now", or "prefer not to say".

--------------------------------------------------
QUESTION POLICY (CRITICAL)
--------------------------------------------------
- Ask EXACTLY ONE question per assistant message.
- Ask ONLY the highest-priority UNFILLED field.
- NEVER ask multiple questions.
- NEVER include explanations, summaries, or lists while fields are unfilled.

--------------------------------------------------
VALIDATION GATE (CRITICAL)
--------------------------------------------------
- A field is COLLECTED ONLY if it passes validation.
- If validation fails, the field remains UNFILLED.
- Invalid input MUST trigger re-asking the SAME question.

VALIDATION RULES:
- Email:
  - Must match format text@text.domain
  - Normalize to lowercase
  - On failure: re-ask

- Year of manufacture:
  - Must be between 1980 and current year
  - Normalize to YYYY
  - On failure: re-ask

- IDV:
  - Accept units: k, l, lakh, c, cr, crore
  - Convert to full numeric value
  - Normalize with commas
  - On failure: re-ask

- Policy expiry:
  - Parse absolute or relative date using current timestamp
  - Normalize to DD/MM/YYYY
  - On failure: re-ask

- Phone:
  - Digits only
  - Length 9–11 digits
  - On failure: re-ask
  - Phone number input is explicitly allowed and MUST NOT be blocked

--------------------------------------------------
IMPLICIT SKIP GUARD (CRITICAL)
--------------------------------------------------
- Validation failure MUST NOT result in skipping a field.
- Invalid input requires re-asking the SAME question.

--------------------------------------------------
MANDATORY QUESTION GUARD
--------------------------------------------------
- Fields requiring explicit answers (PA coverage, claims last year)
  MUST be asked directly.
- Smart inference is NOT allowed for these fields.

--------------------------------------------------
PROGRESS DISPLAY BAN (CRITICAL)
--------------------------------------------------
- Do NOT show progress, partial summaries, field lists, or missing indicators
  until ALL fields are collected or explicitly skipped.

--------------------------------------------------
QUESTION-ONLY TURN LOCK
--------------------------------------------------
- If ANY field is unfilled:
  - Assistant response MUST contain ONLY:
    - Exactly ONE question
    - No explanations
    - No summaries
    - No status text

--------------------------------------------------
SUMMARY & CONFIRMATION
--------------------------------------------------
WHEN ALL FIELDS ARE VALIDATED OR SKIPPED:
- Show normalized summary.
- End summary with:
  "Is this information correct? [[CONFIRM_DETAILS]]"

CONFIRMATION HANDLING:
- Accepted confirmations: Yes, S, Confirm, Correct
- On confirmation:
  Output exactly:
  [QUOTE_COMPLETE]

- On correction:
  Update the corrected field and show the summary again.

--------------------------------------------------
FINAL OUTPUT CONTRACT (CRITICAL)
--------------------------------------------------
- When quote is completed:
  - Output EXACTLY: [QUOTE_COMPLETE]
  - As the FINAL line
  - No extra text
  - No formatting"""

def generate_response(prompt, max_new_tokens=256):
    """Generate a response from the fine-tuned model."""
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": prompt}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        do_sample=True,
        use_cache=True,
    )

    # Decode only new tokens
    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    return response

# Test it!
test_prompts = [
    "Hello! Can you help me?",
    "What can you do?",
]

print("Testing fine-tuned model:")
print("=" * 50)
for prompt in test_prompts:
    print(f"\nUser: {prompt}")
    response = generate_response(prompt)
    print(f"Assistant: {response}")
    print("-" * 50)


## Export Options

Run the cells below to export your model to different formats.


In [ ]:
#@title Export to GGUF (for Ollama, LM Studio) - Optional
#@markdown Check the box to enable export. **Adds ~5-10 min.**
run_gguf_export = False  #@param {type:"boolean"}

if run_gguf_export:
    GGUF_OUTPUT = "./fine_tuned_llama_3.2_3b_gguf"
    QUANT_METHOD = "q4_k_m"  # Good balance of size/quality

    print(f"Exporting to GGUF ({QUANT_METHOD})...")
    model.save_pretrained_gguf(GGUF_OUTPUT, tokenizer, quantization_method=QUANT_METHOD)
    print(f"GGUF model saved to: {GGUF_OUTPUT}")

    import shutil
    shutil.make_archive(GGUF_OUTPUT, 'zip', GGUF_OUTPUT)
    print(f"\nDownload: {GGUF_OUTPUT}.zip")
else:
    print("Skipped GGUF export. Check the box above and re-run if needed.")


In [ ]:
#@title Export Merged Model (for HuggingFace) - Optional
#@markdown Check the box to enable export. **Adds ~3-5 min.**
run_merged_export = False  #@param {type:"boolean"}

if run_merged_export:
    MERGED_OUTPUT = "./fine_tuned_llama_3.2_3b_merged"

    print("Exporting 16-bit merged model...")
    model.save_pretrained_merged(MERGED_OUTPUT + "_16bit", tokenizer, save_method="merged_16bit")
    print(f"Saved to: {MERGED_OUTPUT}_16bit")
else:
    print("Skipped merged export. Check the box above and re-run if needed.")


## Download Your Model

1. **Click the folder icon** in the left sidebar
2. **Navigate to the output folder** (e.g., `fine_tuned_llama_3.2_3b`)
3. **Right-click** on the file/folder > **Download**

### Using Your Model

**With Unsloth (recommended):**
```python
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="./fine_tuned_llama_3.2_3b",
    max_seq_length=2048,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)
```

**With HuggingFace (after merging):**
```python
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("./fine_tuned_llama_3.2_3b_merged_16bit")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_llama_3.2_3b_merged_16bit")
```

**With Ollama (after GGUF export):**
```bash
# Create Modelfile
echo 'FROM ./fine_tuned_llama_3.2_3b_gguf/model.gguf' > Modelfile
ollama create my-model -f Modelfile
ollama run my-model
```

---
*Happy fine-tuning!*
